# Neural Network to Classify Terrain using IMU and Odometer data from TurtleBot3
### By Jack Demeter
After experimenting with the use of logistical regression and KNN we futher explore the classification of the data set with supervised learning on the delta dataset generated in the clustering experimentation

In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

from tensorflow.keras import Sequential
import tensorflow.keras.layers

%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

In [2]:
# Set to your directory
dataFolder = r"C:\Users\jaker\Documents\Experiment3Data-2019-11-21"

dataFiles = [
    r"gMitTile_s15_t8.csv",
    r"gMitTile_s15_t9.csv",
    r"gMitTile_s15_t10.csv",
    r"gTurf_s15_t3.csv",
    r"gTurf_s15_t4.csv",
    r"gTurf_s15_t5.csv",
    r"gTurf_s15_t6.csv",
    r"gTurf_s15_t7.csv",
    r"gTurf_s15_t8.csv",
    r"gTurf_s15_t9.csv",
    r"gTurf_s15_t10.csv",
    r"gArcTile_s15_t3.csv",
    r"gArcTile_s15_t4.csv",
    r"gArcTile_s15_t5.csv",
    r"gArcTile_s15_t6.csv",
    r"gArcTile_s15_t7.csv",
    r"gArcTile_s15_t8.csv",
    r"gArcTile_s15_t9.csv",
    r"gArcTile_s15_t10.csv",
    r"gCarp_s15_t3.csv",
    r"gCarp_s15_t4.csv",
    r"gCarp_s15_t5.csv",
    r"gCarp_s15_t6.csv",
    r"gCarp_s15_t7.csv",
    r"gCarp_s15_t8.csv",
    r"gCarp_s15_t9.csv",
    r"gCarp_s15_t10.csv",
    r"gMitTile_s15_t3.csv",
    r"gMitTile_s15_t4.csv",
    r"gMitTile_s15_t5.csv",
    r"gMitTile_s15_t6.csv",
    r"gMitTile_s15_t7.csv"
]

# Data Loading & Preprocessing

For each data file we:
    adjust the index, 
    interpolate NaN values,
    drop remaining NaN values, 
    drop some empty columns.

We then augment the feature space with delta columns, polynomial columns, and label the columns with terrain, speed, and trial number

In [3]:
for i in range(len(dataFiles)):
    terrain = dataFiles[i].split('_')[0][1:]
    speed = dataFiles[i].split('_s')[1][:2]
    trial = dataFiles[i].split('_t')[1][0]

    df = pd.read_csv(os.path.join(dataFolder, dataFiles[i]))
    df = df.rename(columns={'Unnamed: 0': 'Seq'})
    df = df.set_index('Seq')

    df = df.interpolate(method='polynomial', order=1)

    df = df.dropna()
    df = df.reset_index().drop(columns=['Seq'])

    df = df.drop(columns=['OdomPosZ', 'OdomOrientX', 'OdomOrientY', 'OdomLinY', 'OdomLinZ', 'OdomAngX', 'OdomAngY'])
    df['OdomPosXY'] = np.sqrt(df.OdomPosX**2 + df.OdomPosY**2)
    df = df.drop(columns=['OdomPosY', 'OdomPosX'])
    order=2
    
    dList = range(1, 302, 20)
    for col in df.columns.tolist():
        if col!='Sensor':
            for d in dList:
                df[col+'Delta{}'.format(d)] = df[col].diff(d)
                if order>1:
                    for p in range(2, order+1):
                        df[col+'Delta{}Exp{}'.format(d, p)] = df[col+'Delta{}'.format(d)]**p
        else:
            pass

    df = df.iloc[max(dList):].reset_index().drop(columns=['index'])
    df = df.drop(columns=['Sensor', 'Time'])
    df['Speed']=int(speed)
    df['Terrain']=terrain
    df['Trial']=int(trial)

    if i==0:
        mainDf = df.copy(deep=True)
    else:
        mainDf = pd.concat([mainDf, df], axis=0, sort=False)
print("Data series completed")
print("MainDf is now size {}".format(mainDf.shape))

Added gMitTile_s15_t8.csv of size (3530, 530) to mainDf. 
Data series completed: 1/32
MainDf is now size (3530, 530)


Added gMitTile_s15_t9.csv of size (4030, 530) to mainDf. 
Data series completed: 2/32
MainDf is now size (7560, 530)


Added gMitTile_s15_t10.csv of size (4552, 530) to mainDf. 
Data series completed: 3/32
MainDf is now size (12112, 530)


Added gTurf_s15_t3.csv of size (5800, 530) to mainDf. 
Data series completed: 4/32
MainDf is now size (17912, 530)


Added gTurf_s15_t4.csv of size (5539, 530) to mainDf. 
Data series completed: 5/32
MainDf is now size (23451, 530)


Added gTurf_s15_t5.csv of size (3051, 530) to mainDf. 
Data series completed: 6/32
MainDf is now size (26502, 530)


Added gTurf_s15_t6.csv of size (4952, 530) to mainDf. 
Data series completed: 7/32
MainDf is now size (31454, 530)


Added gTurf_s15_t7.csv of size (4950, 530) to mainDf. 
Data series completed: 8/32
MainDf is now size (36404, 530)


Added gTurf_s15_t8.csv of size (6367, 530) to mainDf. 
D

In [4]:
mainDf.describe(include='all')

,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,OdomPosXY,TimeDelta1,TimeDelta1Exp2,TimeDelta21,TimeDelta21Exp2,TimeDelta41,TimeDelta41Exp2,TimeDelta61,TimeDelta61Exp2,TimeDelta81,TimeDelta81Exp2,TimeDelta101,TimeDelta101Exp2,TimeDelta121,TimeDelta121Exp2,TimeDelta141,TimeDelta141Exp2,TimeDelta161,TimeDelta161Exp2,TimeDelta181,TimeDelta181Exp2,TimeDelta201,TimeDelta201Exp2,TimeDelta221,TimeDelta221Exp2,TimeDelta241,TimeDelta241Exp2,TimeDelta261,TimeDelta261Exp2,TimeDelta281,TimeDelta281Exp2,TimeDelta301,TimeDelta301Exp2,OdomOrientZDelta1,OdomOrientZDelta1Exp2,OdomOrientZDelta21,OdomOrientZDelta21Exp2,OdomOrientZDelta41,OdomOrientZDelta41Exp2,OdomOrientZDelta61,OdomOrientZDelta61Exp2,OdomOrientZDelta81,OdomOrientZDelta81Exp2,OdomOrientZDelta101,OdomOrientZDelta101Exp2,OdomOrientZDelta121,...,ImuAccelZDelta61Exp2,ImuAccelZDelta81,ImuAccelZDelta81Exp2,ImuAccelZDelta101,ImuAccelZDelta101Exp2,ImuAccelZDelta121,ImuAccelZDelta121Exp2,ImuAccelZDelta141,ImuAccelZDelta141Exp2,ImuAccelZDelta161,ImuAccelZDelta161Exp2,ImuAccelZDelta181,ImuAccelZDelta181Exp2,ImuAccelZDelta201,ImuAccelZDelta201Exp2,ImuAccelZDelta221,ImuAccelZDelta221Exp2,ImuAccelZDelta241,ImuAccelZDelta241Exp2,ImuAccelZDelta261,ImuAccelZDelta261Exp2,ImuAccelZDelta281,ImuAccelZDelta281Exp2,ImuAccelZDelta301,ImuAccelZDelta301Exp2,OdomPosXYDelta1,OdomPosXYDelta1Exp2,OdomPosXYDelta21,OdomPosXYDelta21Exp2,OdomPosXYDelta41,OdomPosXYDelta41Exp2,OdomPosXYDelta61,OdomPosXYDelta61Exp2,OdomPosXYDelta81,OdomPosXYDelta81Exp2,OdomPosXYDelta101,OdomPosXYDelta101Exp2,OdomPosXYDelta121,OdomPosXYDelta121Exp2,OdomPosXYDelta141,OdomPosXYDelta141Exp2,OdomPosXYDelta161,OdomPosXYDelta161Exp2,OdomPosXYDelta181,OdomPosXYDelta181Exp2,OdomPosXYDelta201,OdomPosXYDelta201Exp2,OdomPosXYDelta221,OdomPosXYDelta221Exp2,OdomPosXYDelta241,OdomPosXYDelta241Exp2,OdomPosXYDelta261,OdomPosXYDelta261Exp2,OdomPosXYDelta281,OdomPosXYDelta281Exp2,OdomPosXYDelta301,OdomPosXYDelta301Exp2,Speed,Terrain,Trial
count,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,1.453530e+05,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,1.453530e+05,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,...,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.0,145353,145353.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [5]:
# check for none values and correct size
print(mainDf.isnull().sum().sum()) # check for NaN
mainDf.shape

0


(145353, 530)

# Feature Engineering: Elimination and Filtering

In [6]:
# eliminate columns derived from time, and from angular velocity
columnsToDrop = [x for x in mainDf.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Speed' not in x and 'Terrain' not in x and 'Trial' not in x) ]
mainDf = mainDf.drop(columns=columnsToDrop)

In [7]:
# Map tile to numbers, going from "smoothest" to "least smooth" (not that this should have an effect on results)
mainDf['Terrain'] = mainDf['Terrain'].replace({'ArcTile':0,'MitTile':1,'Carp':2,'Turf':3})

In [8]:
mainDf = mainDf[mainDf['Speed']==15]

In [10]:
# number of columns needed in initial neural net layer
len(mainDf.columns)

483

# Neural Network Exploration

In [11]:
X = mainDf.drop(columns=['Terrain', 'Speed', 'Trial'])

In [12]:
y = mainDf['Terrain']

# convert to one hot
OHy = np.zeros((y.size, mainDf['Terrain'].max()+1))
OHy[np.arange(y.size), y] = 1

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [13]:
X.describe(include='all')

,OdomOrientZDelta1,OdomOrientZDelta1Exp2,OdomOrientZDelta21,OdomOrientZDelta21Exp2,OdomOrientZDelta41,OdomOrientZDelta41Exp2,OdomOrientZDelta61,OdomOrientZDelta61Exp2,OdomOrientZDelta81,OdomOrientZDelta81Exp2,OdomOrientZDelta101,OdomOrientZDelta101Exp2,OdomOrientZDelta121,OdomOrientZDelta121Exp2,OdomOrientZDelta141,OdomOrientZDelta141Exp2,OdomOrientZDelta161,OdomOrientZDelta161Exp2,OdomOrientZDelta181,OdomOrientZDelta181Exp2,OdomOrientZDelta201,OdomOrientZDelta201Exp2,OdomOrientZDelta221,OdomOrientZDelta221Exp2,OdomOrientZDelta241,OdomOrientZDelta241Exp2,OdomOrientZDelta261,OdomOrientZDelta261Exp2,OdomOrientZDelta281,OdomOrientZDelta281Exp2,OdomOrientZDelta301,OdomOrientZDelta301Exp2,OdomOrientWDelta1,OdomOrientWDelta1Exp2,OdomOrientWDelta21,OdomOrientWDelta21Exp2,OdomOrientWDelta41,OdomOrientWDelta41Exp2,OdomOrientWDelta61,OdomOrientWDelta61Exp2,OdomOrientWDelta81,OdomOrientWDelta81Exp2,OdomOrientWDelta101,OdomOrientWDelta101Exp2,OdomOrientWDelta121,OdomOrientWDelta121Exp2,OdomOrientWDelta141,OdomOrientWDelta141Exp2,OdomOrientWDelta161,OdomOrientWDelta161Exp2,OdomOrientWDelta181,OdomOrientWDelta181Exp2,OdomOrientWDelta201,OdomOrientWDelta201Exp2,OdomOrientWDelta221,OdomOrientWDelta221Exp2,OdomOrientWDelta241,OdomOrientWDelta241Exp2,OdomOrientWDelta261,OdomOrientWDelta261Exp2,...,ImuAccelZDelta41,ImuAccelZDelta41Exp2,ImuAccelZDelta61,ImuAccelZDelta61Exp2,ImuAccelZDelta81,ImuAccelZDelta81Exp2,ImuAccelZDelta101,ImuAccelZDelta101Exp2,ImuAccelZDelta121,ImuAccelZDelta121Exp2,ImuAccelZDelta141,ImuAccelZDelta141Exp2,ImuAccelZDelta161,ImuAccelZDelta161Exp2,ImuAccelZDelta181,ImuAccelZDelta181Exp2,ImuAccelZDelta201,ImuAccelZDelta201Exp2,ImuAccelZDelta221,ImuAccelZDelta221Exp2,ImuAccelZDelta241,ImuAccelZDelta241Exp2,ImuAccelZDelta261,ImuAccelZDelta261Exp2,ImuAccelZDelta281,ImuAccelZDelta281Exp2,ImuAccelZDelta301,ImuAccelZDelta301Exp2,OdomPosXYDelta1,OdomPosXYDelta1Exp2,OdomPosXYDelta21,OdomPosXYDelta21Exp2,OdomPosXYDelta41,OdomPosXYDelta41Exp2,OdomPosXYDelta61,OdomPosXYDelta61Exp2,OdomPosXYDelta81,OdomPosXYDelta81Exp2,OdomPosXYDelta101,OdomPosXYDelta101Exp2,OdomPosXYDelta121,OdomPosXYDelta121Exp2,OdomPosXYDelta141,OdomPosXYDelta141Exp2,OdomPosXYDelta161,OdomPosXYDelta161Exp2,OdomPosXYDelta181,OdomPosXYDelta181Exp2,OdomPosXYDelta201,OdomPosXYDelta201Exp2,OdomPosXYDelta221,OdomPosXYDelta221Exp2,OdomPosXYDelta241,OdomPosXYDelta241Exp2,OdomPosXYDelta261,OdomPosXYDelta261Exp2,OdomPosXYDelta281,OdomPosXYDelta281Exp2,OdomPosXYDelta301,OdomPosXYDelta301Exp2
count,1.453530e+05,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,1.453530e+05,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,...,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000000,1.453530e+05,145353.000

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

array([[-0.00900217, -0.00981962, -0.00725091, ..., -0.52332287,
        -0.17654227, -0.55871405],
       [-0.00900217, -0.00981962, -0.00812254, ..., -0.523325  ,
        -0.1762811 , -0.55872295],
       [-0.00900217, -0.00981962, -0.00899417, ..., -0.52332712,
        -0.17602083, -0.55873174],
       ...,
       [-0.01364928, -0.00981871, -0.01656127, ...,  0.10822805,
         0.96606246,  0.11722824],
       [-0.01364928, -0.00981871, -0.01683205, ...,  0.10820232,
         0.94882964,  0.09641226],
       [-0.01364928, -0.00981871, -0.01710284, ...,  0.10817658,
         0.94799889,  0.095417  ]])

In [15]:
y = y

## Split data into training and testing 

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, OHy, test_size=0.3)

In [17]:
X.shape

(145353, 480)

## Model Design

We use a Dense network to clasify which frames have potholes in them.

In [45]:
from keras import Sequential
from keras.layers import Dense


classifier = Sequential()
classifier.add(Dense(100, activation='relu', input_dim=480))

classifier.add(Dense(50, activation='relu'))

classifier.add(Dense(4,activation='softmax'))

In [46]:
classifier.compile(optimizer ='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [47]:
np.isnan(y_train).any()

False

## Model Training

In [48]:
classifier.fit(X_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
101747/101747 [==============================] - 2s 18us/step - loss: 0.2672 - categorical_accuracy: 0.8990
Epoch 2/10
101747/101747 [==============================] - 2s 17us/step - loss: 0.0938 - categorical_accuracy: 0.9673
Epoch 3/10
101747/101747 [==============================] - 2s 17us/step - loss: 0.0531 - categorical_accuracy: 0.9817
Epoch 4/10
101747/101747 [==============================] - 2s 17us/step - loss: 0.0365 - categorical_accuracy: 0.9878
Epoch 5/10
101747/101747 [==============================] - 2s 17us/step - loss: 0.0232 - categorical_accuracy: 0.9925
Epoch 6/10
101747/101747 [==============================] - 2s 17us/step - loss: 0.0156 - categorical_accuracy: 0.9949
Epoch 7/10
101747/101747 [==============================] - 2s 17us/step - loss: 0.0175 - categorical_accuracy: 0.9942
Epoch 8/10
101747/101747 [==============================] - 2s 18us/step - loss: 0.0130 - categorical_accuracy: 0.9956
Epoch 9/10
101747/101747 [======================

## Results

In [53]:
_, accuracy =classifier.evaluate(X_train, y_train)
# Output loss and accuracy of model
accuracy

101747/101747 [==============================] - 2s 17us/step


[0.007521706623835023, 0.9975330829620361]

The dense NN can accurately determine which surface the robot was traversing on. We later found out that this was due to improper splitting of the train test dataset. When changing the dataset to not split the same runs between the training and testing the accuracy dropped to 50%, making in no better than a guess.